In [1]:
import ee,os
import geemap

In [2]:
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
ee.Initialize()

Map = geemap.Map(center = [30, 110], zoom = 7)
Map

Map(center=[30, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [3]:
Np = ee.FeatureCollection("users/wufvckshuo/Nepal_Boundary")
ROI = ee.FeatureCollection("users/wufvckshuo/QingJiang")
Np = ROI
L7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
# L7 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

LULC = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")
DEM = ee.Image("USGS/SRTMGL1_003")
L7_vis = {"opacity":1,"bands":["B4","B3","B2"],"min":220,"max":19660,"gamma":1}

L7_np = L7.filterDate('2001-01-01', '2001-12-31') \
.filterBounds(Np) \
.filterMetadata("CLOUD_COVER", 'less_than', 10) \
.sort('CLOUD_COVER',False)

In [4]:
mergedL7 = L7_np.median().clip(Np)

# Map.addLayer(mergedL7, L7_vis, "Landsat 7")
# Map.addLayer(LULC_icimod, LULC_vis, "LULC icimod")

b2 = mergedL7.select("B2")
b3 = mergedL7.select("B3")
b4 = mergedL7.select("B4")
b5 = mergedL7.select("B5")
NDVI = b4.subtract(b3).divide(b4.add(b3)).rename("NDVI")
NDBI = b5.subtract(b4).divide(b5.add(b4)).rename("NDBI")
NDWI = b4.subtract(b5).divide(b5.add(b4)).rename("NDWI")
MNDWI = b2.subtract(b5).divide(b2.add(b5)).rename("MNDWI")
DEM_nepal = DEM.clip(Np)

In [5]:
addedNepal = mergedL7

In [6]:
exportBands = mergedL7.select("B1").addBands(mergedL7.select("B2")) \
.addBands(mergedL7.select("B3")) \
.addBands(mergedL7.select("B4")) \
.addBands(mergedL7.select("B5")) \
.addBands(mergedL7.select("B6")) \
.addBands(mergedL7.select("B7")) \
.addBands(NDVI.select("NDVI")) \
.addBands(NDBI.select("NDBI")) \
.addBands(NDWI.select("NDWI"))\
.addBands(MNDWI.select("MNDWI"))\
.addBands(DEM_nepal) \
.addBands(LULC.first().select("b1").rename("LULC"))

In [7]:
# MAKE SURE ASSET OF RANDOM FOREST <10.4MB is uploaded

In [8]:
from geemap import ml

In [12]:
# read the exported tree feature collection
rf_fc = ee.FeatureCollection("users/wufvckshuo/random_forest_strings_test")

# rf_fc = ee.FeatureCollection("users/wufvckshuo/samplingPoints_ZDQ")

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

In [13]:
# feature names
feature_names = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDWI', 'NDVI', 'MNDWI', 'NDBI', 'elevation']

# classify the image again but with the classifier from the persisted trees
classified = exportBands.select(feature_names).classify(another_classifier)

In [14]:
Map.addLayer(classified, {"min": 0, "max": 6, "palette": ['red', 'green', 'blue']},'classification')

In [15]:
Map

Map(bottom=13819.0, center=[30, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

In [16]:
geemap.ee_export_image_to_drive(classified, description='LULC_MAP_2020_COMP_proper_CLASS', folder='GEE export folder', region=Np.geometry(), file_format='GeoTIFF', format_options={'cloudOptimized': True}, scale=30)


Exporting LULC_MAP_2020_COMP_proper_CLASS ...


In [17]:
geemap.ee_export_image(classified, filename="LULC_MAP_2020.tif", region=Np.geometry(), scale=500)


Generating URL ...
Please wait ...
Data downloaded to e:\Work\LULC-icimod\LULC_MAP_2020.tif
